In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import json
import pickle
from datetime import date, timedelta

In [ ]:
_kamers = open("kamers.json", "r")
kamers = json.loads(_kamers.read())
_kamers.close()

_devices = open("devices.p", "rb")
devices = pickle.load(_devices)
_devices.close()

# Todo: derive from the dataset
years = ['2018', '2019', '2020']

In [ ]:
beneden = devices[kamers[0]]['Temperatuur_gemeten'].filter(items=['value'])
beneden.rename(columns={beneden.columns[0]: 'beneden' }, inplace = True)

boven_1 = devices[kamers[1]]['Temperatuur_gemeten'].filter(items=['value'])
boven_1.rename(columns={boven_1.columns[0]: 'boven_1' }, inplace = True)

boven_2 = devices[kamers[2]]['Temperatuur_gemeten'].filter(items=['value'])
boven_2.rename(columns={boven_2.columns[0]: 'boven_2' }, inplace = True)

boven_3 = devices[kamers[3]]['Temperatuur_gemeten'].filter(items=['value'])
boven_3.rename(columns={boven_3.columns[0]: 'boven_3' }, inplace = True)

boven_4 = devices[kamers[4]]['Temperatuur_gemeten'].filter(items=['value'])
boven_4.rename(columns={boven_4.columns[0]: 'boven_4' }, inplace = True)

temperatuur = beneden.join([boven_1, boven_2, boven_3, boven_4])

temperatuur['mean'] = temperatuur.apply(lambda row: (row['beneden'] * 4 + row['boven_1'] + row['boven_2'] + row['boven_3'] + row['boven_4']) / 8, axis=1)
temperatuur['min'] = temperatuur.min(axis = 1)
temperatuur['max'] = temperatuur.max(axis = 1)

sns.set(rc={'figure.figsize':(18, 6)})

In [ ]:
def compare_years(source, years, x_axis = lambda d: d.index.dayofyear):
    fig, ax = plt.subplots()
    for year in years:
        x = source.loc[year, 'mean']
        x = x.resample('D').mean()
        ax.plot(x_axis(x), x, marker='x', linestyle='-', linewidth=0.5, label=str(year))
    ax.set_ylabel('Gemiddelde temperatuur in graden celcius')
    ax.set_xlabel('Gemiddelde per dag')
    ax.legend()

In [ ]:
compare_years(temperatuur, years)

In [ ]:
df = temperatuur.filter(items=['mean', 'min', 'max'])
df = df.resample('3H').mean()
for y in years:
    ax = df[str(y)].plot.line(y=['max', 'min', 'mean'])
    ax.set_xlabel('Jaar ' + str(y))
    ax.set_ylabel('Temperatuur in celcius')